# Building a Catalog

In [3]:
from classes.source_catalog import GetCnilCatalog

url = 'https://www.data.gouv.fr/api/1/organizations/534fff61a3a7292c64a77d59/catalog'
headers = {'accept': 'application/json'}
url_add = 'https://www.data.gouv.fr/fr/organizations/cnil/datasets.csv'
instance1 = GetCnilCatalog(url, headers, url_add)
data = instance1.fetch_data_from_api()
data = data['@graph']
table_name = 'title'
download_url = 'downloadURL'
table_id = 'identifier' 
file_format= 'format'
last_update= 'modified' 
accessURL = '@id'
df_catalog = instance1.response_to_dataframe(data=data, table_name=table_name, download_url=download_url, table_id=table_id, file_format=file_format, last_update=last_update, accessURL=accessURL)
df_dataset = instance1.load_additional_info()
df_catalog = instance1.identify_datasets_info()
df_catalog = instance1.merge_additional_info()
instance1.save_to_csv('source_cnil_catalog')

Request is a success: 200
CSV file has been loaded to this path data/catalog/source_cnil_catalog_2024-02-15.csv


# Uploading Files to GCS

In [5]:
from classes.file_to_gcs import FromFileToGCS
import os

bucket_name = 'cnil_csv'
cred_path = 'cred/service_account_local_py.json'
init2 = FromFileToGCS(bucket_name, cred_path)
init2.create_bucket()
file_paths = ['data/catalog/source_cnil_catalog_2024-02-10.csv']
dest_folder = 'raw'
dest_blob = ['source_cnil_catalog_2024-02-15.csv']
init2.local_to_gcs(file_paths, dest_folder, dest_blob)

Bucket already exists.
Raw file source_cnil_catalog_2024-02-15.csv uploaded to GCS successfully to 2024-02-15/raw/source_cnil_catalog_2024-02-15.csv.


In [6]:
bucket_name = 'cnil_csv'
cred_path = 'cred/service_account_local_py.json'
init2 = FromFileToGCS(bucket_name, cred_path)
init2.create_bucket()
url = ['https://www.data.gouv.fr/fr/organizations/cnil/datasets.csv', 'https://www.data.gouv.fr/fr/datasets/r/0f678674-4327-4c4d-8819-b6f508b41d0e']
dest_folder = 'raw'
dest_blob = ['datasets.csv', 'plaintes.csv']
init2.download_and_upload_from_URLs(url, dest_folder, dest_blob)

Bucket already exists.
Raw file datasets.csv downloaded and uploaded to GCS successfully to 2024-02-15/raw/datasets.csv.
Raw file plaintes.csv downloaded and uploaded to GCS successfully to 2024-02-15/raw/plaintes.csv.


# Downloading from Catalog

In [7]:
from classes.download_catalog_content import DlCatalogContent

instance3 = DlCatalogContent('data/catalog/source_cnil_catalog_2024-02-15.csv')
instance3.get_tables()
instance3.zip_files()

Error when downloading table Organismes ayant désigné un(e) délégué(e) à la protection des données (DPD/DPO) : Invalid URL 'nan': No scheme supplied. Perhaps you meant http://nan?
Error when downloading table Notifications à la CNIL de violations de données à caractère personnel : Invalid URL 'nan': No scheme supplied. Perhaps you meant http://nan?
Error when downloading table Sanctions prononcées par la CNIL : Invalid URL 'nan': No scheme supplied. Perhaps you meant http://nan?
Error when downloading table Traitements de données personnelles déclarés à la CNIL depuis le 25 mai 2018 : Invalid URL 'nan': No scheme supplied. Perhaps you meant http://nan?
Error when downloading table Protection des données personnelles dans le monde : Invalid URL 'nan': No scheme supplied. Perhaps you meant http://nan?
Error when downloading table Contrôles réalisés par la CNIL : Invalid URL 'nan': No scheme supplied. Perhaps you meant http://nan?
Error when downloading table Mises en demeure prononcées p

# Prep data to upload to BQ

In [1]:
from classes.file_to_gcs import FromFileToGCS
import os

bucket_name = 'cnil_csv'
cred_path = 'cred/service_account_local_py.json'
init2 = FromFileToGCS(bucket_name, cred_path)
init2.create_bucket()
file_paths = ['data/raw_datasets.zip']
dest_folder = 'raw' 
init2.local_to_gcs(file_paths, dest_folder)

Bucket already exists.
Raw file raw_datasets.zip uploaded to GCS successfully to 2024-02-15/raw/raw_datasets.zip.


In [1]:
from classes.prep_data import ZipFileProcessor

gcs_bucket_name = 'cnil_csv'
credential_path = 'cred/service_account_local_py.json'
zip_blob_name = '2024-02-15/raw/raw_datasets.zip'
output_folder_name = '2024-02-15/'+ 'prep'
instance4 = ZipFileProcessor(gcs_bucket_name, credential_path, zip_blob_name, output_folder_name)
zip_file = instance4.get_zip_file_object()

In [3]:
import pandas as pd
from classes.prep_data import PrepDataCnilBQ

# file_list = [file for file in zip_file.namelist()]
# filtered_list = list(filter(lambda x: not x.endswith('/'), file_list))
# filtered_list
# for file in filtered_list:
#   with zip_file.open(file) as file:
#       print(file)
#       if file.name == '.DS_Store':
#         continue
#       try:
#         df = pd.read_csv(file)
#         file.seek(0) 
#         df = pd.read_csv(file, sep=';')
#       except:
#         df = pd.read_excel(file)

#       print(df)

# df 

instance5 = PrepDataCnilBQ(zip_file)
zip_output = instance5.process_zip_file(zip_file)

current: .DS_Store
---------------------------------------------------
.DS_Store
.DS_Store not processed!
---------------------------------------------------
current: budget-de-la-cnil-1/opencnil-budget-depuis-2000-maj-juin-2023_v2023-06-28.csv
---------------------------------------------------
budget-de-la-cnil-1/opencnil-budget-depuis-2000-maj-juin-2023_v2023-06-28.csv
<zipfile.ZipExtFile name='budget-de-la-cnil-1/opencnil-budget-depuis-2000-maj-juin-2023_v2023-06-28.csv' mode='r' compress_type=deflate>
budget-de-la-cnil-1/opencnil-budget-depuis-2000-maj-juin-2023_v2023-06-28.csv
.csv found
file: <zipfile.ZipExtFile name='budget-de-la-cnil-1/opencnil-budget-depuis-2000-maj-juin-2023_v2023-06-28.csv' mode='r' compress_type=deflate>
path: budget-de-la-cnil-1/opencnil-budget-depuis-2000-maj-juin-2023_v2023-06-28.csv
(6, 1)
columns shape is 1, csv read with ;
<class 'zipfile.ZipExtFile'>
opened df, return from open_df
this is df
More columns than rows, need to transpose
The column_forma

/Users/benjamindupaquier/anaconda3/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


(496, 6)
(496, 6)
opened df, return from open_df
this is df
More rows than columns, no need to transpose
The column_formatter method worked perfectly.
The column_formatter method worked perfectly.
The column_formatter method worked perfectly.
The column_formatter method worked perfectly.
The column_formatter method worked perfectly.
The column_formatter method worked perfectly.
Re-exécution terminée.
controles-realises-par-la-cnil/Liste des contrôles réalisés par la CNIL en 2015_v2016-05-03 processed successfully!
---------------------------------------------------
current: controles-realises-par-la-cnil/opencnil-liste-controles-2019_v2020-11-13.xlsx
---------------------------------------------------
controles-realises-par-la-cnil/opencnil-liste-controles-2019_v2020-11-13.xlsx
<zipfile.ZipExtFile name='controles-realises-par-la-cnil/opencnil-liste-controles-2019_v2020-11-13.xlsx' mode='r' compress_type=deflate>
controles-realises-par-la-cnil/opencnil-liste-controles-2019_v2020-11-13.x

/Users/benjamindupaquier/anaconda3/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


(430, 7)
(430, 7)
opened df, return from open_df
this is df
More rows than columns, no need to transpose
The column_formatter method worked perfectly.
The column_formatter method worked perfectly.
The column_formatter method worked perfectly.
The column_formatter method worked perfectly.
The column_formatter method worked perfectly.
The column_formatter method worked perfectly.
Re-exécution terminée.
controles-realises-par-la-cnil/Liste des contrôles réalisés par la CNIL en 2016_v2017-03-30 processed successfully!
---------------------------------------------------
current: controles-realises-par-la-cnil/open-data-controles-2020-vd-20210603_v2021-06-03.xlsx
---------------------------------------------------
controles-realises-par-la-cnil/open-data-controles-2020-vd-20210603_v2021-06-03.xlsx
<zipfile.ZipExtFile name='controles-realises-par-la-cnil/open-data-controles-2020-vd-20210603_v2021-06-03.xlsx' mode='r' compress_type=deflate>
controles-realises-par-la-cnil/open-data-controles-20

/Users/benjamindupaquier/anaconda3/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


(96524, 26)
try to find headers in 2nd row
opened df, return from open_df
this is df
More rows than columns, no need to transpose
The column_formatter method worked perfectly.
The column_formatter method worked perfectly.
The column_formatter method worked perfectly.
The column_formatter method worked perfectly.
The column_formatter method worked perfectly.
The column_formatter method worked perfectly.
The column_formatter method worked perfectly.
The column_formatter method worked perfectly.
The column_formatter method worked perfectly.
The column_formatter method worked perfectly.
The column_formatter method worked perfectly.
The column_formatter method worked perfectly.
The column_formatter method worked perfectly.
The column_formatter method worked perfectly.
The column_formatter method worked perfectly.
The column_formatter method worked perfectly.
The column_formatter method worked perfectly.
The column_formatter method worked perfectly.
The column_formatter method worked perfect

In [4]:
import zipfile
with zipfile.ZipFile(zip_output, 'r') as temp_zip:
    infolist = temp_zip.infolist()
    with temp_zip.open(infolist[0]) as file:
        file_name = file.name
        file_name = file.name.replace('-', '_')
        print(file_name)

budget_de_la_cnil_1/opencnil_budget_depuis_2000_maj_juin_2023_v2023_06_28.csv
